# Creating Pandapower Networks

This Tutorial will introduce the user into the pandapower datastructure and how to create networks through the pandapower API. The following minimal example contains the most commont elements that are supported by the Pandapower format. For an example that contains all pandapower elements (3 winding transformers, ward equivalents, impedances), see the [advanced tutorial](create_advanced.ipynb) for network creation.

<img src="pics/example_network_simple.png">

The datastructure of the pandapower framework is based on the python library pandas. A pandapower network consist of a separate element table for each element type that is used in the network. Each element table consists of a column for each parameter and a row for each element. By executing the follwing code cells you generate the various element tables.  You can find detailed descriptions about each parameter in the pandapower documentation under bulletpoint "Datastructures and Elements".

### Empty Network

First, we import pandapower an create an empty network:

In [1]:
from pandapower.create import *

net = create_empty_network() #create an empty network

### Buses

<img src="pics/example_network_simple_buses.png">

We now create the three high voltage (vn_kv=110.) and four medium voltage (vn_kv=20.) buses.

In [2]:
bus1 = create_bus(net, name="HV Busbar", vn_kv=110, type="b")
bus2 = create_bus(net, name="HV Busbar 2", vn_kv=110, type="b")
bus3 = create_bus(net, name="HV Transformer Bus", vn_kv=110, type="n")
bus4 = create_bus(net, name="MV Transformer Bus", vn_kv=20, type="n")
bus5 = create_bus(net, name="MV Main Bus", vn_kv=20, type="b")
bus6 = create_bus(net, name="MV Bus 1", vn_kv=20, type="b")
bus7 = create_bus(net, name="MV Bus 2", vn_kv=20, type="b")

Bus 3 and bus 4 are classified as nodes (type="n"), all other bus types are declared as busbars (type="b"):

In [3]:
net.bus # show bus table

,name,vn_kv,type,zone,in_service,geo
0,HV Busbar,110.0,b,None,True,None
1,HV Busbar 2,110.0,b,None,True,None
2,HV Transformer Bus,110.0,n,None,True,None
3,MV Transformer Bus,20.0,n,None,True,None
4,MV Main Bus,20.0,b,None,True,None
5,MV Bus 1,20.0,b,None,True,None
6,MV Bus 2,20.0,b,None,True,None


All create functions return the pandapower index of the element that was created, for example the variable bus6 is now equal to the index of the bus with the name "MV Station 2" (which is 5):

In [4]:
bus6

np.int64(5)

We use these variables for creating bus and branch elements in the following.

### External Grid

<img src="pics/example_network_simple_ext_grid.png">

We now create an external grid connection that serves as slack node for the power flow calculation. The voltage of the external grid is set to a magnitude of 1.02 per unit and 50 degrees voltage angle:

In [5]:
create_ext_grid(net, bus1, vm_pu=1.02, va_degree=50) # Create an external grid connection

net.ext_grid #show external grid table

,name,bus,vm_pu,va_degree,slack_weight,in_service,controllable
0,None,0,1.02,50.0,1.0,True,False


### Transformer 

<img src="pics/example_network_simple_trafo.png">

The transformer connects the high-voltage with the medium-voltage side of the grid. The high-voltage bus of the transformer is connected to Bus3 and on the medium-voltage side the transformer is linked to Bus4. We select the standard type "25 MVA 110/20 kV" from the pandapower basic standard type library:

In [6]:
trafo1 = create_transformer(net, bus3, bus4, name="110kV/20kV transformer", std_type="25 MVA 110/20 kV")

 The detailled transformer parameters, such as short circuit voltages, rated power or iron losses are automatically loaded from the standard type library (see [standard type library tutorial](std_types.ipynb)) and stored in the transformer table:

In [7]:
net.trafo #show transformer table

,name,std_type,hv_bus,lv_bus,sn_mva,vn_hv_kv,vn_lv_kv,vk_percent,vkr_percent,pfe_kw,...,tap_step_degree,tap_pos,tap_changer_type,id_characteristic_table,tap_dependency_table,parallel,df,in_service,vector_group,oltc
0,110kV/20kV transformer,25 MVA 110/20 kV,2,3,25.0,110.0,20.0,12.0,0.41,14.0,...,0.0,0.0,Ratio,<NA>,False,1,1.0,True,YNd5,False


### Lines

The network includes three medium voltage lines and one high voltage line. The bus connections and line lengths are defined in the network diagram:

<img src="pics/example_network_simple_lines.png">

The line parameters are once again taken from the standard type library (see [standard type library tutorial](std_types.ipynb)). We use different line lengths and standard types for each line:

In [8]:
line1 = create_line(net, bus1, bus2, length_km=10, std_type="N2XS(FL)2Y 1x300 RM/35 64/110 kV",  name="Line 1")
line2 = create_line(net, bus5, bus6, length_km=2.0, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 2")
line3 = create_line(net, bus6, bus7, length_km=3.5, std_type="48-AL1/8-ST1A 20.0", name="Line 3")
line4 = create_line(net, bus7, bus5, length_km=2.5, std_type="NA2XS2Y 1x240 RM/25 12/20 kV", name="Line 4")

The full line table looks like this:

In [9]:
net.line # show line table

,name,std_type,from_bus,to_bus,length_km,r_ohm_per_km,x_ohm_per_km,c_nf_per_km,g_us_per_km,max_i_ka,df,parallel,type,in_service,geo
0,Line 1,N2XS(FL)2Y 1x300 RM/35 64/110 kV,0,1,10.0,0.0600,0.144,144.0,0.0,0.588,1.0,1,cs,True,None
1,Line 2,NA2XS2Y 1x240 RM/25 12/20 kV,4,5,2.0,0.1220,0.112,304.0,0.0,0.421,1.0,1,cs,True,None
2,Line 3,48-AL1/8-ST1A 20.0,5,6,3.5,0.5939,0.372,9.5,0.0,0.210,1.0,1,ol,True,None
3,Line 4,NA2XS2Y 1x240 RM/25 12/20 kV,6,4,2.5,0.1220,0.112,304.0,0.0,0.421,1.0,1,cs,True,None


### Switches

There are two circuit breakers on the high- and low voltage side of the transformer which connect two buses (Bus2-Bus3 and Bus4-Bus5). These bus-bus switches can be defined with et="b". 
<img src="pics/example_network_simple_switches.png">

In [10]:
sw1 = create_switch(net, bus2, bus3, et="b", type="CB", closed=True)
sw2 = create_switch(net, bus4, bus5, et="b", type="CB", closed=True)

Furthermore, we equip all bus/line connections in the medium voltage level with load break switches ("LBS") as shown in the network diagram. Bus/Line switches are defined with et="l":

In [11]:
sw3 = create_switch(net, bus5, line2, et="l", type="LBS", closed=True)
sw4 = create_switch(net, bus6, line2, et="l", type="LBS", closed=True)
sw5 = create_switch(net, bus6, line3, et="l", type="LBS", closed=True)
sw6 = create_switch(net, bus7, line3, et="l", type="LBS", closed=False)
sw7 = create_switch(net, bus7, line4, et="l", type="LBS", closed=True)
sw8 = create_switch(net, bus5, line4, et="l", type="LBS", closed=True)

The switch table now shows all switches. The bus colum contains the index of the bus the switch is connected to. For bus switches (et="b"), the element column contains the index of the second bus the switch connects to. For line switches (et="l"), the element column contains the index of the line the switch connects to. All switches are closed.

In [12]:
net.switch # show switch table

,bus,element,et,type,closed,name,z_ohm,in_ka
0,1,2,b,CB,True,None,0.0,NaN
1,3,4,b,CB,True,None,0.0,NaN
2,4,1,l,LBS,True,None,0.0,NaN
3,5,1,l,LBS,True,None,0.0,NaN
4,5,2,l,LBS,True,None,0.0,NaN
5,6,2,l,LBS,False,None,0.0,NaN
6,6,3,l,LBS,True,None,0.0,NaN
7,4,3,l,LBS,True,None,0.0,NaN


### Load

<img src="pics/example_network_simple_load.png">

The load element is used to model by default constant active and reactive power consumption. We create a 2 MW / 4 MVar load with a scaling factor of 0.6:

In [13]:
create_load(net, bus7, p_mw=2, q_mvar=4, scaling=0.6, name="load")

net.load

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
0,load,6,2.0,4.0,0.0,0.0,0.0,0.0,NaN,0.6,True,wye


#### Voltage dependent loads - ZIP load model
One can observe load parameters `const_z_percent` and `const_i_percent` which are set to 0 by default. They can be used in order to define part of a load which is voltage-dependent using the so-called **ZIP load** model, which allows a load definition as a composition of constant power, constant current and constant impedance.  

As an example, we create a 2 MW / 4 MVar load with 30% of constant impedance and 20% of constant current:

In [14]:
create_load(net, bus7, p_mw=2, q_mvar=4, const_z_percent=30, const_i_percent=20, name="zip_load")

net.load

,name,bus,p_mw,q_mvar,const_z_p_percent,const_i_p_percent,const_z_q_percent,const_i_q_percent,sn_mva,scaling,in_service,type
0,load,6,2.0,4.0,0.0,0.0,0.0,0.0,NaN,0.6,True,wye
1,zip_load,6,2.0,4.0,30.0,20.0,30.0,20.0,NaN,1.0,True,wye


### Static Generator

<img src="pics/example_network_simple_sgen.png">

The static generator element is used to model constant active and reactive power generation. Since the [signing system](https://pandapower.readthedocs.io/en/latest/_images/p_q_reference_system.png) used in pandapower for generation units is from a generator's point of view, the active power has to be positive to model generation. We create a static generator with 2 MW generation and 500 kVar.

In [15]:
create_sgen(net, bus7, p_mw=2, q_mvar=-0.5, name="static generator")

net.sgen

,name,bus,p_mw,q_mvar,min_q_mvar,max_q_mvar,sn_mva,scaling,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,type,current_source
0,static generator,6,2.0,-0.5,NaN,NaN,NaN,1.0,False,<NA>,False,NaN,True,wye,True


### Voltage controlled Generator

<img src="pics/example_network_simple_gen.png">

The generator element is used to model voltage controlled active power generation. We define it with a active power generation (positive) and a voltage set point:

In [16]:
create_gen(net, bus6, p_mw=6, max_q_mvar=3, min_q_mvar=-3, vm_pu=1.03, name="generator") 

net.gen

,name,bus,p_mw,vm_pu,sn_mva,min_q_mvar,max_q_mvar,scaling,slack,controllable,id_q_capability_characteristic,reactive_capability_curve,curve_style,in_service,slack_weight,type
0,generator,5,6.0,1.03,NaN,-3.0,3.0,1.0,False,True,<NA>,False,NaN,True,0.0,None


### Shunt


<img src="pics/example_network_simple_shunt.png">

The shunt is defined by its active and reactive power consumption at rated voltage. Once again, the signing system is from a consumers point of view. We want to model a a capacitator bank, and therefore have to assign a negative reactive power to the shunt:

In [17]:
create_shunt(net, bus3, q_mvar=-0.96, p_mw=0, name='Shunt')

net.shunt

,bus,name,q_mvar,p_mw,vn_kv,step,max_step,id_characteristic_table,step_dependency_table,in_service
0,2,Shunt,-0.96,0.0,110.0,1,1,<NA>,False,True


If you want to learn how to create more complicated networks, continue with the [advanced create tutorial](create_advanced.ipynb). If you want to learn about how to run a loadflow, continue with the [power flow tutorial](powerflow.ipynb). 